<a href="https://colab.research.google.com/github/shinjeongdong/MLDeeplearningStudy/blob/main/%5BPyTorch%5D/%5BPyTorch%5D_5_2_%EB%82%98%EB%A7%8C%EC%9D%98_%EB%AA%A8%EB%93%88.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#import

In [ ]:
#주요 함수를 모듈화 해서 가독성과 코드를 줄임
import torch
from torch import nn, optim
from torchvision import datasets, transforms
import numpy as np
import matplotlib.pyplot as plt

import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks/func/')
from multiclass_functions1 import *

DEVICE = 'cuda' if torch.cuda.is_available() else "cpu"
print(DEVICE)

cpu


In [ ]:
BATCH_SIZE = 32
LR = 1e-3
EPOCH = 5
criterion = nn.CrossEntropyLoss()
new_model_train = False
model_type = 'MLP'
dataset = "MNSIT"
save_model_path = f"/content/drive/MyDrive/Colab Notebooks/results/{model_type}.pt"

In [ ]:
transform = transforms.ToTensor()
train_DS = datasets.MNIST(root = "/content/drive/MyDrive/Colab Notebooks", train=True, download= True, transform=transform)
test_DS = datasets.MNIST(root = "/content/drive/MyDrive/Colab Notebooks", train=False, download= True, transform=transform)
train_DL = torch.utils.data.DataLoader(train_DS, batch_size = BATCH_SIZE, shuffle = True) # 32개 무작위
test_DL = torch.utils.data.DataLoader(test_DS, batch_size = BATCH_SIZE, shuffle = True) # 32개 무작위

In [ ]:
class MLP(nn.Module):
  def __init__(self):
    super().__init__()
    self.fcs = nn.Sequential(nn.Linear(28*28, 100),
                             nn.ReLU(),
                             nn.Linear(100,10)) #짜피 로스를 ce로 잡으면 자동으로 소프트맥스가 됨

  def forward(self,x):
    x = torch.flatten(x,start_dim = 1)
    x = self.fcs(x)
    return x

In [ ]:
model = globals()[model_type]().to(DEVICE)
print(model)
x_batch,_ = next(iter(train_DL))
print(x_batch.shape)
print(model(x_batch.to(DEVICE)).shape) #로짓

MLP(
  (fcs): Sequential(
    (0): Linear(in_features=784, out_features=100, bias=True)
    (1): ReLU()
    (2): Linear(in_features=100, out_features=10, bias=True)
  )
)
torch.Size([32, 1, 28, 28])
torch.Size([32, 10])


In [ ]:
if new_model_train:
  optimizer = optim.Adam(model.parameters(), lr = LR)
  loss_history = Train(model, train_DL, criterion, optimizer, EPOCH)

  torch.save(model,save_model_path)

  plt.plot(range(1,EPOCH + 1),loss_history)
  plt.xlabel("EPOCH")
  plt.ylabel("loss")
  plt.title("Train loss")
  plt.grid()

In [ ]:
load_model = torch.load(save_model_path,map_location=DEVICE, weights_only= False)

In [ ]:
Test(load_model,test_DL)

Test accuracy: 9747/10000 (97.5 %)


#모듈 함수

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
DEVICE = "cuda" if torch.cuda.is_available() else 'cpu'

def Train(model,train_DL,criterion, optimizer, EPOCH):

  loss_history = [] #로스 저장용 리스트
  Not = len(train_DL.dataset) #60000

  model.train() #학습모드
  for ep in range(EPOCH):
    rloss = 0 #running loss
    for x_batch, y_batch in train_DL: #60000개를 32,1,28,28로 꺼내서 학습함 미니배치 학습
      x_batch = x_batch.to(DEVICE) # 32,1,28,28
      y_batch = y_batch.to(DEVICE)
      #inference 추론
      y_hat = model(x_batch) #x_batch 32개를 넣고 y_hat값을 얻음
      #loss
      loss = criterion(y_hat,y_batch) #그 y_hat이랑 y정답값을 이용해 로스 손실 얻음

      #update
      optimizer.zero_grad() #gd 누적 막기위한 초기화
      loss.backward() #역전파 : W에 대한 편미분
      optimizer.step() #업데이트
      #loss accumulation
      loss_b = loss.item() * x_batch.shape[0] #로스랑 32를곱함 왜? CE는 자동으로 평균값을 나타냄 고로 다시 32를 곱해서 전체합을 나타냄
      rloss += loss_b #그것을 rloss에다 누적함 다 누적하면 60000개 대한 로스가 더해짐
    #print loss
    loss_e = rloss/Not # 그걸 60000개로 나눠준다 그러면 전체 EPOCH에 대한 로스 평균이 구해짐
    loss_history += [loss_e]
    print(f"Epoch : {ep + 1}, train_loss : {loss_e:.3f}")
    print("-"*20)
  return loss_history


def Test(model, test_DL):

  model.eval() #eval no_grad는 외우자
  with torch.no_grad():
    rcorrect = 0 #맞춘개수 누적
    for x_batch,y_batch in test_DL:
      x_batch = x_batch.to(DEVICE) #32,1,28,28
      y_batch = y_batch.to(DEVICE) #(32)
      #inference
      y_hat = model(x_batch) #y_hat 구하고 (32,10)
      #corrects accumulation
      pred = y_hat.argmax(dim = 1) #32행에서 가로축 원소들 중에 가장 큰값들의 인덱스만 꺼내옴 (32) #1차원
      corrects_b = torch.sum(pred == y_batch).item() # 예측값이랑 정답값이랑 일치하는지
      #print(pred)

      rcorrect += corrects_b #누적시킴

    accuracy_e = rcorrect / len(test_DL.dataset) * 100 # 그러면 (정답수 / 전체 데이터 수)해서 정답률 가능

  print(f"Test accuracy: {rcorrect}/{len(test_DL.dataset)} ({accuracy_e:.1f} %)")


def Test_plot(model,test_DL):
  model.eval()
  with torch.no_grad():
    x_batch, y_batch = next(iter(test_DL))
    x_batch = x_batch.to(DEVICE)
    y_hat = model(x_batch)
    pred = y_hat.argmax(dim = 1)

  x_batch = x_batch.to('cpu')


  plt.figure(figsize = (8,4))
  for idx in range(6):
    plt.subplot(2,3, idx+1,xticks = [], yticks=[])
    plt.imshow(x_batch[idx].permute(1,2,0).squeeze(), cmap='gray')
    pred_class = test_DL.dataset.classes[pred[idx]]
    true_class = test_DL.dataset.classes[y_batch[idx]]
    plt.title(f"{pred_class} ({true_class})",color = 'g' if pred_class == true_class else "r")


def count_params(model):
  num = sum([p.numel() for p in model.parameters() if p.requires_grad]) #numel = 행렬의 원소의 개수 리턴
  return num


